In [ ]:
# ---------------------------
# Blockchain builder 
# ---------------------------
import json, hashlib, uuid, os, tempfile
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Any

HOME = Path("/home/jovyan")

# Paths 
FILES = {
    "video": HOME / "Video_analysis+IOT" / "outputs" / "video_anomalies.json",
    "iot": HOME / "Video_analysis+IOT" / "outputs" / "iot_anomalies.json",
    "prohibited": HOME / "CLCXray_yolo" / "outputs" / "prohibited_items.json",
}

LEDGER_PATH = HOME / "security_ledger.json"

#  Helpers 
def sha256_hex(s: str) -> str:
    return hashlib.sha256(s.encode("utf-8")).hexdigest()

def atomic_write(path: Path, data: str) -> None:
    d = path.parent
    d.mkdir(parents=True, exist_ok=True)
    fd, tmp = tempfile.mkstemp(prefix=path.name, dir=str(d))
    try:
        with os.fdopen(fd, "w") as f:
            f.write(data)
        os.replace(tmp, str(path))
    finally:
        if os.path.exists(tmp):
            try: os.remove(tmp)
            except: pass

def load_json_file(p: Path) -> List[Dict[str,Any]]:
    if not p.exists():
        # no file is okay — return empty
        return []
    try:
        with open(p, "r") as f:
            data = json.load(f)
    except Exception as e:
        print(f"⚠️ Failed to read {p}: {e}")
        return []
    # normalize shapes
    if isinstance(data, list):
        return data
    if isinstance(data, dict):
        # common patterns: {"detections": [...] } or dict of filename->list
        if "detections" in data and isinstance(data["detections"], list):
            return data["detections"]
        # if dict of lists, flatten
        flattened = []
        for v in data.values():
            if isinstance(v, list):
                flattened.extend(v)
        if flattened:
            return flattened
        # otherwise wrap single dict as list
        return [data]
    return []

def extract_field(entry: Dict[str,Any], keys):
    for k in keys:
        if k in entry and entry[k] is not None:
            return entry[k]
    return None

def normalize_event(entry: Dict[str,Any], source: str) -> Dict[str,Any]:
    # Build a normalized event dict for the ledger
    # event_id: deterministic if possible (filename+timestamp+class) else UUID
    filename = extract_field(entry, ["filename","file","image","img","path"])
    ts = extract_field(entry, ["timestamp","time","datetime"]) or (datetime.utcnow().isoformat()+"Z")
    item_class = extract_field(entry, ["item_class","class","label","anomaly_type","type","name"]) or "unknown"
    # score priority: anomaly_score -> confidence -> score -> recon_error
    score = extract_field(entry, ["anomaly_score","confidence","score","recon_error"])
    try:
        score = float(score) if score is not None else 0.0
    except:
        score = 0.0
    # details: keep small useful keys
    details = {}
    for k in ["details","index","frame","bbox","annotations","info"]:
        if k in entry:
            details[k] = entry[k]
    if filename: details["filename"] = filename

    # create deterministic event id when possible
    if filename or ts or item_class:
        fingerprint = f"{filename}|{ts}|{item_class}"
        event_id = sha256_hex(fingerprint)[:32]
    else:
        event_id = str(uuid.uuid4())

    return {
        "event_id": event_id,
        "timestamp": ts,
        "source": source,
        "item_class": str(item_class),
        "anomaly_score": float(round(score, 6)),
        "details": details
    }

# Collect events from all files 
def collect_all_events(paths_map=FILES) -> List[Dict[str,Any]]:
    events = []
    for src, pth in paths_map.items():
        p = Path(pth)
        data = load_json_file(p)
        if not data:
            # print(f"⚠️ No data found for {src} at {p}")
            continue
        for entry in data:
            ev = normalize_event(entry, src)
            events.append(ev)
    # sort by timestamp for deterministic ordering
    try:
        events.sort(key=lambda x: x["timestamp"])
    except Exception:
        pass
    return events

#  Build chain from events list 
def build_chain(events: List[Dict[str,Any]]) -> List[Dict[str,Any]]:
    chain = []
    prev_hash = "0"
    for idx, ev in enumerate(events, start=1):
        block = {
            "index": idx,
            "event_id": ev["event_id"],
            "timestamp": ev["timestamp"],
            "source": ev["source"],
            "item_class": ev.get("item_class","unknown"),
            "anomaly_score": ev.get("anomaly_score", 0.0),
            "details": ev.get("details", {}),
            "prev_hash": prev_hash
        }
        # compute hash over sorted keys (without hash)
        block_str = json.dumps(block, sort_keys=True)
        block_hash = sha256_hex(block_str)
        block["hash"] = block_hash
        chain.append(block)
        prev_hash = block_hash
    return chain

# Save ledger atomically 
def save_ledger(chain: List[Dict[str,Any]], path: Path = LEDGER_PATH) -> None:
    atomic_write(path, json.dumps(chain, indent=4))
    print(f"✅ Ledger saved → {path} ({len(chain)} blocks)")

# Verify ledger integrity 
def verify_ledger(path: Path = LEDGER_PATH) -> bool:
    if not path.exists():
        print("❌ Ledger file not found.")
        return False
    try:
        with open(path, "r") as f:
            chain = json.load(f)
    except Exception as e:
        print("❌ Failed to read ledger:", e)
        return False
    prev_hash = "0"
    for b in chain:
        copy_block = {k: b[k] for k in b if k != "hash"}
        recalculated = sha256_hex(json.dumps(copy_block, sort_keys=True))
        if recalculated != b.get("hash"):
            print(f"❌ Hash mismatch at index {b.get('index')}")
            return False
        if b.get("prev_hash") != prev_hash:
            print(f"❌ prev_hash mismatch at index {b.get('index')}")
            return False
        prev_hash = b.get("hash")
    print(f"✅ Ledger verified: {len(chain)} blocks OK.")
    return True

# Build & save ledger (rebuild mode overwrites) 
def build_and_save_ledger(rebuild: bool = True) -> List[Dict[str,Any]]:
    events = collect_all_events(FILES)
    if not events:
        print("⚠️ No events found in input files. Ledger not updated.")
        return []
    # build chain anew (rebuild) or append to existing
    if rebuild or not LEDGER_PATH.exists():
        chain = build_chain(events)
        save_ledger(chain, LEDGER_PATH)
    else:
        # append mode: read existing and append only new event_ids
        with open(LEDGER_PATH, "r") as f:
            existing = json.load(f)
        existing_ids = {b.get("event_id") for b in existing}
        new_events = [e for e in events if e["event_id"] not in existing_ids]
        if not new_events:
            print("✅ No new events to append.")
            return existing
        start_idx = len(existing) + 1
        prev_hash = existing[-1]["hash"] if existing else "0"
        new_blocks = []
        for i, ev in enumerate(new_events):
            block = {
                "index": start_idx + i,
                "event_id": ev["event_id"],
                "timestamp": ev["timestamp"],
                "source": ev["source"],
                "item_class": ev.get("item_class","unknown"),
                "anomaly_score": ev.get("anomaly_score",0.0),
                "details": ev.get("details", {}),
                "prev_hash": prev_hash
            }
            block["hash"] = sha256_hex(json.dumps(block, sort_keys=True))
            new_blocks.append(block)
            prev_hash = block["hash"]
        full_chain = existing + new_blocks
        save_ledger(full_chain, LEDGER_PATH)
        chain = full_chain
    return chain

# Quick preview helper 
def preview_ledger(n=10, path: Path = LEDGER_PATH):
    if not path.exists():
        print("❌ Ledger not found.")
        return
    with open(path, "r") as f:
        chain = json.load(f)
    for b in chain[:n]:
        print(f"{b['index']:>4} | {b['source']:<10} | {b['item_class']:<18} | score: {b['anomaly_score']:<6} | time: {b['timestamp']}")

# ---------------------------
# RUN: build ledger now (rebuild -> overwrite)
# ---------------------------
chain = build_and_save_ledger(rebuild=True)

# verify and preview
if chain:
    verify_ledger()
    print("\n🔎 Preview first 12 blocks:")
    preview_ledger(12)
    # print counts per source
    from collections import Counter
    src_counts = Counter([b["source"] for b in chain])
    print("\n📌 Counts per source:", dict(src_counts))


✅ Ledger saved → /home/jovyan/security_ledger.json (1930 blocks)
✅ Ledger verified: 1930 blocks OK.

🔎 Preview first 12 blocks:
   1 | video      | Crowd Movement     | score: 0.004062 | time: 2025-11-12 08:04:10.977407
   2 | video      | Crowd Movement     | score: 0.004302 | time: 2025-11-12 08:04:10.977434
   3 | video      | Crowd Movement     | score: 0.004546 | time: 2025-11-12 08:04:10.977437
   4 | video      | Crowd Movement     | score: 0.004793 | time: 2025-11-12 08:04:10.977439
   5 | video      | Crowd Movement     | score: 0.004971 | time: 2025-11-12 08:04:10.977442
   6 | video      | Crowd Movement     | score: 0.005081 | time: 2025-11-12 08:04:10.977444
   7 | video      | Crowd Movement     | score: 0.005124 | time: 2025-11-12 08:04:10.977447
   8 | video      | Crowd Movement     | score: 0.005176 | time: 2025-11-12 08:04:10.977451
   9 | video      | Crowd Movement     | score: 0.005243 | time: 2025-11-12 08:04:10.977454
  10 | video      | Crowd Movement     | sco